In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-05-18 04:31:43.389681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 04:31:43.498061: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-18 04:31:44.042934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [ ]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
def generate_split(fake_cols, file_path):

    
    loader = LJDataLoader(data_path=file_path)
    loader.sample(0.1)
    loader.splitData()

    source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
    new_col_name = 'RandWaveFake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
    new_col_name = 'Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

    #data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
    data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=fake_cols)
    
    train_count = data_df[data_df['type'] == 'train'].shape[0]
    dev_count = data_df[data_df['type'] == 'dev'].shape[0]
    test_count = data_df[data_df['type'] == 'test'].shape[0]

    print(f'# of Train instances: {train_count}')
    print(f'# of Dev instances: {dev_count}')
    print(f'# of Test instances: {test_count}')
    
    return data_df

In [ ]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    #cadence_manager = CadenceModelManager(data_df)
    #cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline() # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary')
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass')
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    #feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss']
    results = pd.DataFrame(columns=results_cols)
    
    for label_type in ['label', 'multiclass_label']:
        for k, v in feature_store.items():
            model_manager = ModelManager('logreg', v[0], v[1], merge_train_dev=True)
            model_manager.trainPredict(label_col=label_type)
            print(model_manager.class_accuracy)
            results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name):
    data_df = generate_split(fake_cols, metadata_path)
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    results.to_csv(f'/home/ubuntu/data/results/{name}.csv', index=False)
    
    

In [7]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['ElevenLabs'], file_path, '16KHz_ElevenLabs')

# of Train instances: 1566
# of Dev instances: 522
# of Test instances: 524
[NeMo I 2023-05-18 04:31:48 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:31:48 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:31:48 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:31:49 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:31:49 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:31:49 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:31:49 features:267] PADDING: 16
[NeMo I 2023-05-18 04:31:49 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:24<00:00, 12.13s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:32:23 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:32:23 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:32:23 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning:


Selected features:.

1. pcm_fftMag_spectralEntropy_sma_de_lpgain
2. pcm_fftMag_spectralEntropy_sma_de_range
3. pcm_fftMag_spectralEntropy_sma_de_lpc1
4. audspec_lengthL1norm_sma_de_peakMeanRel
5. audSpec_Rfilt_sma[5]_qregc2
6. pcm_fftMag_spectralEntropy_sma_de_kurtosis
7. pcm_fftMag_spectralEntropy_sma_linregc2
8. pcm_fftMag_spectralEntropy_sma_upleveltime75
9. pcm_fftMag_spectralEntropy_sma_upleveltime90
10. pcm_fftMag_spectralSlope_sma_de_peakMeanRel
{0: 1.0, 1: 1.0}
{0: 0.9541984732824428, 1: 0.9694656488549618}
{0: 1.0, 1: 1.0}
{0: 1.0, 1: 1.0}
{0: 0.9541984732824428, 1: 0.9694656488549618}
{0: 1.0, 1: 1.0}


For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss

In [8]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['ElevenLabs'], file_path, '16KHz_ElevenLabs')

# of Train instances: 1566
# of Dev instances: 522
# of Test instances: 524
[NeMo I 2023-05-18 04:32:24 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:32:24 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:32:24 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:32:24 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:32:24 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:32:24 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:32:24 features:267] PADDING: 16
[NeMo I 2023-05-18 04:32:25 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:23<00:00, 11.82s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:32:58 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:32:58 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:32:58 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning:


Selected features:.

1. mfcc_sma_de[14]_rqmean
2. pcm_fftMag_spectralEntropy_sma_de_maxPos
3. pcm_fftMag_spectralEntropy_sma_skewness
4. pcm_fftMag_spectralEntropy_sma_upleveltime50
5. pcm_fftMag_spectralEntropy_sma_de_lpc2
6. pcm_fftMag_spectralEntropy_sma_de_lpc0
7. pcm_fftMag_spectralEntropy_sma_upleveltime75
8. pcm_fftMag_spectralEntropy_sma_minRangeRel
9. pcm_fftMag_spectralEntropy_sma_upleveltime90
10. pcm_RMSenergy_sma_de_peakMeanRel
{0: 1.0, 1: 1.0}
{0: 0.9198473282442748, 1: 0.9732824427480916}
{0: 1.0, 1: 0.9923664122137404}
{0: 1.0, 1: 1.0}
{0: 0.9198473282442748, 1: 0.9732824427480916}
{0: 1.0, 1: 0.9923664122137404}


In [9]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck')

# of Train instances: 2349
# of Dev instances: 783
# of Test instances: 786
[NeMo I 2023-05-18 04:32:58 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:32:58 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:32:58 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:32:59 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:32:59 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:32:59 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:32:59 features:267] PADDING: 16
[NeMo I 2023-05-18 04:32:59 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:36<00:00, 12.07s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:33:52 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:33:52 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:33:52 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklear


Selected features:.

1. pcm_fftMag_spectralEntropy_sma_de_maxPos
2. mfcc_sma_de[8]_lpgain
3. audspec_lengthL1norm_sma_minPos
4. pcm_fftMag_spectralEntropy_sma_de_lpc1
5. mfcc_sma_de[13]_lpgain
6. pcm_fftMag_spectralEntropy_sma_upleveltime75
7. pcm_fftMag_spectralEntropy_sma_kurtosis
8. pcm_RMSenergy_sma_de_peakMeanRel
9. mfcc_sma[13]_lpgain
10. mfcc_sma_de[14]_meanFallingSlope
{0: 0.9885496183206107, 1: 1.0}
{0: 0.6679389312977099, 1: 0.9064885496183206}
{0: 0.9427480916030534, 1: 0.9599236641221374}
{0: 1.0, 1: 1.0, 2: 1.0}


[NeMo W 2023-05-18 04:33:52 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
      n_iter_i = _check_optimize_result(
    
[NeMo W 2023-05-18 04:33:52 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],

{0: 0.7099236641221374, 1: 0.7519083969465649, 2: 0.9656488549618321}
{0: 0.9580152671755725, 1: 0.9732824427480916, 2: 0.9809160305343512}


In [10]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix')

# of Train instances: 3132
# of Dev instances: 1044
# of Test instances: 1048
[NeMo I 2023-05-18 04:33:53 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:33:53 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:33:53 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:33:53 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:33:53 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:33:53 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:33:53 features:267] PADDING: 16
[NeMo I 2023-05-18 04:33:54 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [01:58<00:00, 11.89s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:36:19 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:36:19 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:36:19 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklear


Selected features:.

1. pcm_fftMag_spectralEntropy_sma_de_range
2. audspec_lengthL1norm_sma_de_peakMeanRel
3. pcm_fftMag_spectralEntropy_sma_qregerrQ
4. pcm_fftMag_spectralEntropy_sma_de_lpc0
5. jitterDDP_sma_quartile1
6. pcm_fftMag_spectralEntropy_sma_range
7. jitterDDP_sma_percentile1.0
8. mfcc_sma[9]_lpgain
9. mfcc_sma[14]_quartile3
10. pcm_fftMag_spectralEntropy_sma_kurtosis
{0: 0.5343511450381679, 1: 0.9987277353689568}
{0: 0.2480916030534351, 1: 0.94529262086514}
{0: 0.5419847328244275, 1: 0.9134860050890585}
{0: 0.9656488549618321, 1: 1.0, 2: 0.9961832061068703, 3: 0.9580152671755725}


[NeMo W 2023-05-18 04:36:19 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
      n_iter_i = _check_optimize_result(
    
[NeMo W 2023-05-18 04:36:19 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],

{0: 0.5343511450381679, 1: 0.9694656488549618, 2: 0.7022900763358778, 3: 0.24045801526717558}
{0: 0.6679389312977099, 1: 0.9923664122137404, 2: 0.8816793893129771, 3: 0.5458015267175572}


In [11]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['ElevenLabs'], file_path, '16KHz_ElevenLabs_Laundered')

# of Train instances: 1566
# of Dev instances: 522
# of Test instances: 522
[NeMo I 2023-05-18 04:36:19 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:36:19 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:36:19 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:36:20 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:36:20 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:36:20 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:36:20 features:267] PADDING: 16
[NeMo I 2023-05-18 04:36:20 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:23<00:00, 11.92s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:36:56 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:36:56 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:36:56 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning:


Selected features:.

1. audspecRasta_lengthL1norm_sma_linregc1
2. audSpec_Rfilt_sma[4]_linregc1
3. audspecRasta_lengthL1norm_sma_qregc2
4. audspecRasta_lengthL1norm_sma_qregc1
5. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
6. audSpec_Rfilt_sma[5]_qregc2
7. audspecRasta_lengthL1norm_sma_de_maxPos
8. pcm_fftMag_fband250-650_sma_de_peakMeanRel
9. pcm_fftMag_spectralSlope_sma_upleveltime90
10. pcm_fftMag_spectralSlope_sma_percentile99.0
{0: 0.9540229885057471, 1: 0.9272030651340997}
{0: 0.6513409961685823, 1: 0.7432950191570882}
{0: 0.9003831417624522, 1: 0.8467432950191571}
{0: 0.9540229885057471, 1: 0.9272030651340997}
{0: 0.6513409961685823, 1: 0.7432950191570882}
{0: 0.9003831417624522, 1: 0.8467432950191571}


In [12]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['UberDuck'], file_path, '16KHz_UberDuck_Laundered')

# of Train instances: 1566
# of Dev instances: 522
# of Test instances: 522
[NeMo I 2023-05-18 04:36:56 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:36:56 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:36:56 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:36:57 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:36:57 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:36:57 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:36:57 features:267] PADDING: 16
[NeMo I 2023-05-18 04:36:57 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:24<00:00, 12.13s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:37:34 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:37:34 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:37:34 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklear


Selected features:.

1. mfcc_sma[14]_peakMeanRel
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma[5]_lpc2
4. mfcc_sma[14]_rqmean
5. mfcc_sma_de[13]_meanFallingSlope
6. mfcc_sma_de[13]_meanRisingSlope
7. mfcc_sma[11]_lpc2
8. mfcc_sma[14]_quartile1
9. mfcc_sma[14]_quartile2
10. mfcc_sma[13]_lpgain
{0: 0.8850574712643678, 1: 0.9923371647509579}
{0: 0.5823754789272031, 1: 0.6896551724137931}
{0: 0.8122605363984674, 1: 0.8314176245210728}
{0: 0.8850574712643678, 1: 0.9923371647509579}
{0: 0.5823754789272031, 1: 0.6896551724137931}


[NeMo W 2023-05-18 04:37:34 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
      n_iter_i = _check_optimize_result(
    
[NeMo W 2023-05-18 04:37:34 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],

{0: 0.8122605363984674, 1: 0.8314176245210728}


In [13]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck_Laundered')

# of Train instances: 2349
# of Dev instances: 783
# of Test instances: 783
[NeMo I 2023-05-18 04:37:34 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:37:34 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:37:34 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:37:35 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:37:35 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:37:35 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:37:35 features:267] PADDING: 16
[NeMo I 2023-05-18 04:37:35 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:36<00:00, 12.01s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



[NeMo W 2023-05-18 04:38:29 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:38:29 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:38:29 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklear


Selected features:.

1. audSpec_Rfilt_sma_de[5]_minPos
2. audSpec_Rfilt_sma[4]_linregc1
3. audspec_lengthL1norm_sma_de_peakMeanRel
4. pcm_fftMag_spectralSlope_sma_upleveltime90
5. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
6. audSpec_Rfilt_sma[5]_qregc2
7. mfcc_sma_de[13]_meanFallingSlope
8. audspecRasta_lengthL1norm_sma_qregc2
9. pcm_RMSenergy_sma_de_peakMeanRel
10. pcm_fftMag_spectralSlope_sma_percentile99.0
{0: 0.8199233716475096, 1: 1.0}
{0: 0.2720306513409962, 1: 0.9310344827586207}
{0: 0.4789272030651341, 1: 0.8659003831417624}


[NeMo W 2023-05-18 04:38:29 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:38:29 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

{0: 0.8888888888888888, 1: 0.9501915708812261, 2: 0.9042145593869731}
{0: 0.4827586206896552, 1: 0.5287356321839081, 2: 0.5900383141762452}
{0: 0.6206896551724138, 1: 0.6551724137931034, 2: 0.8045977011494253}


In [14]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix_Laundered')

# of Train instances: 3132
# of Dev instances: 1044
# of Test instances: 1044
[NeMo I 2023-05-18 04:38:30 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-18 04:38:30 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-18 04:38:30 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-18 04:38:30 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-18 04:38:30 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-18 04:38:30 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-18 04:38:30 features:267] PADDING: 16
[NeMo I 2023-05-18 04:38:31 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:02<00:00, 12.21s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_range
3. audspec_lengthL1norm_sma_iqr2-3
4. duration(seconds)
5. audspec_lengthL1norm_sma_quartile3
6. audspec_lengthL1norm_sma_iqr1-3
7. audspec_lengthL1norm_sma_iqr1-2
8. audspec_lengthL1norm_sma_quartile2
9. audspec_lengthL1norm_sma_quartile1
10. audspec_lengthL1norm_sma_maxPos
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...


Selected features:.

1. mfcc_sma[11]_peakDistStddev
2. audSpec_Rfilt_sma[5]_qregc1
3. audSpec_Rfilt_sma[19]_centroid
4. audSpec_Rfilt_sma[5]_maxPos
5. audSpec_Rfilt_sma[5]_qregc2
6. audSpec_Rfilt_sma[5]_centroid
7. pcm_fftMag_spectralSlope_sma_percentile99.0
8. jitterDDP_sma_quartile1
9. mfcc_sma[14]_peakMeanAbs
10. audspecRasta_lengthL1norm_sma_centroid
{0: 0.23754789272030652, 1: 0.9961685823754789}


[NeMo W 2023-05-18 04:40:59 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:41:00 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    


{0: 0.034482758620689655, 1: 0.9897828863346104}
{0: 0.13026819923371646, 1: 0.9642401021711366}


[NeMo W 2023-05-18 04:41:00 nemo_logging:349] /tmp/ipykernel_1578709/405606096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
[NeMo W 2023-05-18 04:41:00 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

{0: 0.7816091954022989, 1: 0.9118773946360154, 2: 0.9425287356321839, 3: 0.8467432950191571}
{0: 0.3946360153256705, 1: 0.5555555555555556, 2: 0.42528735632183906, 3: 0.19157088122605365}
{0: 0.5057471264367817, 1: 0.8467432950191571, 2: 0.6513409961685823, 3: 0.28735632183908044}
